In [33]:
# 기본 라이브러리 불러오기 & EDA 시각화 & 데이터 로드 

In [92]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

In [93]:
train_df = pd.read_csv('C:/Users/wnrrh/Desktop/dacon/택배_운송량_예측대회/train_df.csv',encoding='cp949')
test_df = pd.read_csv('C:/Users/wnrrh/Desktop/dacon/택배_운송량_예측대회/test_df.csv',encoding='cp949')
ss = pd.read_csv('C:/Users/wnrrh/Desktop/dacon/택배_운송량_예측대회/sample_submission.csv',encoding='cp949')
train_df.set_index('index',inplace=True)
test_df.set_index('index',inplace=True)

In [94]:
freq = train_df.groupby(['SEND_SPG_INNB','REC_SPG_INNB']).count()
freq

DL_GD_LCLS_NM  DL_GD_MCLS_NM  INVC_CONT
SEND_SPG_INNB    REC_SPG_INNB                                             
1111000015066100 5011000216039300              1              1          1
1111000017015300 5013000865018300              1              1          1
1111000019090400 5011000083098400              1              1          1
                 5011000113044300              1              1          1
                 5011000313078100              1              1          1
...                                          ...            ...        ...
5013000946025300 4684000253071200              1              1          1
5013000946029100 4117100021020100              1              1          1
                 4117300015071200              1              1          1
                 4278000584065400              1              1          1
5013000957045100 4136000358037300              1              1          1

[31697 rows x 3 columns]

In [95]:
# 격자코드로는 의미 없는 분류인 것 확인
train_df.drop(['SEND_SPG_INNB','REC_SPG_INNB'],axis=1,inplace=True)
test_df.drop(['SEND_SPG_INNB','REC_SPG_INNB'],axis=1,inplace=True)

In [96]:
# 카테고리 개수 확인
freq_item = train_df.groupby(['DL_GD_LCLS_NM','DL_GD_MCLS_NM']).count()
freq_item

INVC_CONT
DL_GD_LCLS_NM DL_GD_MCLS_NM           
디지털/가전        기타디지털/가전             241
생활/건강         건강용품                 176
              문구/사무용품              180
              반려동물                 358
              생활용품                 343
              음반                   635
              주방용품                 328
식품            가공식품                 923
              건강식품                 651
              과자                   348
              기타식품                 766
              냉동/간편조리식품            413
              농산물                22162
              수산                   931
              음료                  1126
              축산                   418
여행/문화         문화컨텐츠               1192
패션의류          기타패션의류               261
              상의                   321
화장품/미용        스킨케어                 227

In [97]:
# 테스트 카테고리 개수 확인
freq_item_test = test_df.groupby(['DL_GD_LCLS_NM','DL_GD_MCLS_NM']).count()
freq_item_test

Empty DataFrame
Columns: []
Index: [(디지털/가전, 기타디지털/가전), (생활/건강, 건강용품), (생활/건강, 문구/사무용품), (생활/건강, 반려동물), (생활/건강, 생활용품), (생활/건강, 음반), (생활/건강, 주방용품), (식품, 가공식품), (식품, 건강식품), (식품, 과자), (식품, 기타식품), (식품, 냉동/간편조리식품), (식품, 농산물), (식품, 수산), (식품, 음료), (식품, 축산), (여행/문화, 문화컨텐츠), (패션의류, 기타패션의류), (패션의류, 상의), (화장품/미용, 스킨케어)]

In [102]:
cate_L_values = test_df['DL_GD_LCLS_NM'].value_counts()
cate_M_values = test_df['DL_GD_MCLS_NM'].value_counts()
print(cate_L_values)
print('--------------------')
cate_M_values

--------------------


농산물          3239
문화컨텐츠         171
음료            167
수산            144
가공식품          134
기타식품          107
건강식품           88
음반             85
축산             61
냉동/간편조리식품      55
생활용품           54
과자             52
상의             47
주방용품           47
반려동물           36
기타패션의류         35
기타디지털/가전       34
스킨케어           29
문구/사무용품        28
건강용품           27
Name: DL_GD_MCLS_NM, dtype: int64

 # EDA 시각화

In [ ]:
df = train_df.sort_values(by=['INVC_CONT'],ascending=[False])
df

# Encoding

In [71]:
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
import numpy as np

encoder_M = LabelEncoder()
train_df['DL_GD_MCLS_NM'] = encoder_M.fit_transform(train_df['DL_GD_MCLS_NM'])

encoder_L = LabelEncoder()
train_df['DL_GD_LCLS_NM'] = encoder_L.fit_transform(train_df['DL_GD_LCLS_NM'])

Train = pd.get_dummies(train_df, columns = ['DL_GD_LCLS_NM', 'DL_GD_MCLS_NM'])

In [72]:
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
import numpy as np

encoder_M = LabelEncoder()
test_df['DL_GD_MCLS_NM'] = encoder_M.fit_transform(test_df['DL_GD_MCLS_NM'])

encoder_L = LabelEncoder()
test_df['DL_GD_LCLS_NM'] = encoder_L.fit_transform(test_df['DL_GD_LCLS_NM'])

Test = pd.get_dummies(test_df, columns = ['DL_GD_LCLS_NM', 'DL_GD_MCLS_NM'])

In [73]:
# Test 셋에 운송량 컬럼 추가
Test['INVC_CONT'] = 0

# 트레이닝 / 테스트 구분

In [74]:
x_train = Train.iloc[:,1:]
y_train = Train['INVC_CONT']
x_test = Test.drop(['INVC_CONT'],axis=1)
y_test = Test.iloc[::,-1]


In [75]:
# 트레이닝, 테스트 구분 비율 8:2
from sklearn.model_selection import train_test_split
x_train,test_x,y_traint,test_y = train_test_split(x_train,y_train,test_size=0.2,shuffle=True)

In [76]:
x_train

,DL_GD_LCLS_NM_0,DL_GD_LCLS_NM_1,DL_GD_LCLS_NM_2,DL_GD_LCLS_NM_3,DL_GD_LCLS_NM_4,DL_GD_LCLS_NM_5,DL_GD_MCLS_NM_0,DL_GD_MCLS_NM_1,DL_GD_MCLS_NM_2,DL_GD_MCLS_NM_3,...,DL_GD_MCLS_NM_10,DL_GD_MCLS_NM_11,DL_GD_MCLS_NM_12,DL_GD_MCLS_NM_13,DL_GD_MCLS_NM_14,DL_GD_MCLS_NM_15,DL_GD_MCLS_NM_16,DL_GD_MCLS_NM_17,DL_GD_MCLS_NM_18,DL_GD_MCLS_NM_19
index,,,,,,,,,,,,,,,,,,,,,
9170,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11087,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22126,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27106,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18161,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9824,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28957,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27731,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
